# A python solution to calculate Pv for pure substances using PR-EOS

***
**Note: Click on the `Cell` in the task bar above, then click on `Run All` to run the entire code**
---
**If you open this on myBinder, then click on `Run` above and then click on `Run all cells`**


In [1]:
import ipywidgets as widgets
import pandas as pd
from IPython.display import display, HTML, Markdown
import math
import copy
from functools import reduce

from library import render_table, render_multi_index_2D_table
import util

## The cells below solves question 1
Calculate the vapor pressure of propane together with the densities of the liquid and gas at 104°F.
Compare your answers with values from Figures 2-7, 2-12, and 3-4 (McCain Jr., W.D. The
Properties of Petroleum Fluids (2nd Edition). PennWell Publishing Company, Tulsa, OK, 1990.). 

### Working data for question one
This will contain the list of species to be considered, their critical properties and acentric factor

In [2]:
# temperatures are in °F and need to be converted to °R
specie = { "name": "propane", "pc": 616.0, "tc": util.to_rankine(206.06, "f"), "w": 0.1522, "Mw": 49.097 }
chart_data = { "zg": [0.75], "zl": [0.05], "pressure": [200], "ρg": [0.03], "ρl": [0.475] }

In [3]:
# Some widgets
temperature = widgets.FloatText(
    value="104", 
    description="Farenheit", 
    layout={'width': '25%', 'margin': '10px 0px 10px 10px'},
)
pressure_widget = widgets.FloatText(
    value="100", 
    min=14.7, 
    description="psia", 
    layout={'width': '25%', 'margin': '10px 0px 10px 10px'},
)

# interact with widgets
def on_value_change(sys_temp, initial_trial_pressure):
    temp = util.to_rankine(sys_temp, 'f')
    
    selected_specie = specie
    pc = selected_specie['pc']
    tc = selected_specie['tc']
    tr = util.calculate_tr(temp, tc)
    b = util.calculate_b(tc, pc)
    ac = util.calculate_ac(tc, pc)
    w = selected_specie['w']
    alpha = util.calculate_alpha(w, temp, tc)
    aT = util.calculate_at(ac, alpha)
    
#     create table for the initially calculated values
    initial_calulated_values = {"Tr": [tr],"b": [b], "ac": [ac], "alpha": [alpha], "at": [aT] }
    display(HTML('<h3>PR-EOS REQUIRED COEFFICIENTS</h3>'))
    display(pd.DataFrame(initial_calulated_values))
    
#     iteration code begins
    iteration_table_data = {"zg": [],"zl": [],"fg": [],"fl": [],"fg - fl": [],"pressure": [],"Vmg": [],"Vml": [],"ρg": [],"ρl": [],}
    
    fg = 0
    fl = 1000
    pressure = initial_trial_pressure
    tolerance = 0.01
    while abs(fg - fl) >= tolerance:
        A = round(util.calculate_A(aT, pressure, temp), 5)
        B = round(util.calculate_B(b, pressure, temp), 6)
        roots = util.cubic_root_calculation_for_PR_EOS(A, B)

        z_gas = util.get_highest_or_lowest_root(roots, 'gas')
        z_liquid = util.get_highest_or_lowest_root(roots, 'liquid')
        iteration_table_data["zg"].append(round(z_gas, 4))
        iteration_table_data["zl"].append(round(z_liquid, 4))

        fg = util.solve_for_fg_fl(z_gas, B, A, pressure)
        fl = util.solve_for_fg_fl(z_liquid, B, A, pressure)
        iteration_table_data["fg"].append(round(fg, 4))
        iteration_table_data["fl"].append(round(fl, 4))
        
        iteration_table_data["pressure"].append(round(pressure, 4))
        iteration_table_data["fg - fl"].append(round(abs(fg - fl), 4))
        
        Vmg = (z_gas * 10.732 * temp) / pressure
        Vml = (z_liquid * 10.732 * temp) / pressure
        iteration_table_data["Vmg"].append(round(Vmg, 4))
        iteration_table_data["Vml"].append(round(Vml, 4))
        
        den_g = ( (pressure * selected_specie["Mw"]) / (z_gas * 10.73 * temp)) / 62.42796
        iteration_table_data["ρg"] = den_g
        
        den_l = ( (pressure * selected_specie["Mw"]) / (z_liquid * 10.73 * temp)) / 62.42796
        iteration_table_data["ρl"] = den_l
        
        pressure = pressure + 0.01
     
    
    display(HTML('<h3>The Iteration results</h3>'))
    iteration_df = pd.DataFrame(iteration_table_data)
    display(iteration_df)
    calculated_data = iteration_df.tail(1)
    display(HTML('<h3>Calculated results</h3>'))
    display(calculated_data)
    display(HTML('<h3>Reading from the charts</h3>'))
    display(pd.DataFrame(chart_data))
    
    
    
# Display widgets 
trap_return = widgets.interact(
    on_value_change, 
    sys_temp=temperature, 
    initial_trial_pressure=pressure_widget
)

interactive(children=(FloatText(value=104.0, description='Farenheit', layout=Layout(margin='10px 0px 10px 10px…